In [11]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.font_manager import FontProperties

# 獲取當前工作目錄
current_dir = os.getcwd()
version3_path = os.path.join(current_dir, "TrafficTDApython", "Version3", "tdamapper", "core_old.py")

from utils.models import *
from utils.utils_v3 import *
from utils.plots import *

try:
    myfont = FontProperties(fname=r"/System/Library/Fonts/PingFang.ttc")
    sns.set(style="whitegrid", font=myfont.get_name())
except Exception as e:
    print(e)

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

dataA2 = pd.read_csv("./Data/A2.csv", low_memory=False)
dataA1 = pd.read_csv("./Data/A1.csv")

[Errno 2] No such file or directory: 'C:\\System\\Library\\Fonts\\PingFang.ttc'


In [18]:
# car data
car_data = pd.read_csv(os.path.join(current_dir, "Data/CarDataNew/full_1.csv"), encoding='utf-8')
motor_data = pd.read_csv(os.path.join(current_dir, "Data/CarDataNew/full_2.csv"), encoding='utf-8')
overlap_data = pd.read_csv(os.path.join(current_dir, "Data/CarDataNew/full_out.csv"), encoding='utf-8')
outlier_data = pd.read_csv(os.path.join(current_dir, "Data/CarDataNew/overlap_data.csv"), encoding='utf-8')
# pass data
pass_data0 = pd.read_csv(os.path.join(current_dir, "Data/PassData/full_0.csv"), encoding='utf-8')
pass_data1 = pd.read_csv(os.path.join(current_dir, "Data/PassData/full_1.csv"), encoding='utf-8')
pass_data2 = pd.read_csv(os.path.join(current_dir, "Data/PassData/full_2.csv"), encoding='utf-8')
pass_data3 = pd.read_csv(os.path.join(current_dir, "Data/PassData/full_3.csv"), encoding='utf-8')
pass_data_overlap = pd.read_csv(os.path.join(current_dir, "Data/PassData/overlap_data.csv"), encoding='utf-8')
pass_data_out = pd.read_csv(os.path.join(current_dir, "Data/PassData/full_out.csv"), encoding='utf-8')

# 合併模型輸入資料，總共有 5 種資料
# car_data
# motor_data
car_motor_outlier_overlap = pd.concat([outlier_data, overlap_data], axis=0)
pass_data = pd.concat([pass_data0, pass_data1, pass_data2, pass_data3, pass_data_overlap], axis=0)

# 補回當事者行動狀態大類別名稱
car_data['當事者行動狀態大類別名稱'] = '車的狀態'
motor_data['當事者行動狀態大類別名稱'] = '車的狀態'
car_motor_outlier_overlap['當事者行動狀態大類別名稱'] = '車的狀態'
pass_data['當事者行動狀態大類別名稱'] = '人的狀態'
pass_data_out['當事者行動狀態大類別名稱'] = '人的狀態'

full_data = pd.concat([car_data, motor_data, car_motor_outlier_overlap, pass_data, pass_data_out], axis=0)
car_data.drop(columns=['當事者行動狀態大類別名稱'], inplace=True)
motor_data.drop(columns=['當事者行動狀態大類別名稱'], inplace=True)
car_motor_outlier_overlap.drop(columns=['當事者行動狀態大類別名稱'], inplace=True)
pass_data.drop(columns=['當事者行動狀態大類別名稱'], inplace=True)

In [19]:
select_lst = [
    '天候名稱', '光線名稱', 
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    
    # 路面狀況
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    # 車道
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    # 當事人
    '當事者屬-性-別名稱', '當事者事故發生時年齡',
    '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱',
    '肇事逃逸類別名稱-是否肇逃',

    # 大類別
    # '道路型態大類別名稱', '事故位置大類別名稱',
    # '車道劃分設施-分向設施大類別名稱',
    # '事故類型及型態大類別名稱', '當事者區分-類別-大類別名稱-車種',
    # '車輛撞擊部位大類別名稱-最初', 
    # '車輛撞擊部位大類別名稱-其他',

    # 兩個欄位只有兩個觀察值不同
    '肇因研判大類別名稱-主要',
    # '肇因研判大類別名稱-個別',
    '受傷', '死亡',
    
    # 子類別
    '道路型態子類別名稱', '事故位置子類別名稱', '事故類型及型態子類別名稱', '肇因研判子類別名稱-主要',
    '當事者區分-類別-子類別名稱-車種', '當事者行動狀態子類別名稱', '車輛撞擊部位子類別名稱-最初',
    '車輛撞擊部位子類別名稱-其他', '肇因研判子類別名稱-個別'
]

# select data
full_data = full_data[select_lst]
car_data = car_data[select_lst]
motor_data = motor_data[select_lst]
car_motor_outlier_overlap = car_motor_outlier_overlap[select_lst]
# select_lst.remove('車輛撞擊部位大類別名稱-最初')
select_lst.remove('行動電話或電腦或其他相類功能裝置名稱')
# select_lst.remove('當事者區分-類別-大類別名稱-車種')
pass_data = pass_data[select_lst]

## 唯一值處理

In [29]:
for data in [car_data, motor_data, car_motor_outlier_overlap, pass_data]:
    columns_to_drop = []
    for column in car_data.columns:
        if car_data[column].nunique() == 1:
            columns_to_drop.append(column)
    data.drop(columns=columns_to_drop, inplace=True)

In [30]:
full_data = pd.get_dummies(full_data)
car_data = pd.get_dummies(car_data)
motor_data = pd.get_dummies(motor_data)
car_motor_outlier_overlap = pd.get_dummies(car_motor_outlier_overlap)
pass_data = pd.get_dummies(pass_data)
pass_data_outlier = pd.get_dummies(pass_data_out)

full_data_X, full_data_y = get_train_test_data(full_data)
car_data_X, car_data_y = get_train_test_data(car_data)
motor_data_X, motor_data_y = get_train_test_data(motor_data)
car_motor_outlier_overlap_X, car_motor_outlier_overlap_y = get_train_test_data(car_motor_outlier_overlap)
pass_data_X, pass_data_y = get_train_test_data(pass_data)
pass_data_outlier_X, pass_data_outlier_y = get_train_test_data(pass_data_outlier)

assert car_data_X.shape[0] + pass_data_X.shape[0] + motor_data_X.shape[0] + car_motor_outlier_overlap_X.shape[0] + pass_data_outlier_X.shape[0] == full_data_X.shape[0]

ModelPerformance 為不使用kfold大類別 <br/>
ModelPerformanceV2 為不使用kfold子類別納入 <br/>
ModelPerformanceNew 為kfold+大類別模型 <br/>
ModelPerformanceNewV2 為kfold+子類別納入 <br/>
ModelPerformanceNewV3 為kfold+只使用子類別 <br/>

In [32]:
def linear_svc_kfold(X, y, k=5, random_state=42, n_jobs=12):
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)

    # 存真實以及decision score方便後續對混淆的計算以及AUC
    y_true_all = []
    decision_scores_all = []
    # 這裡存儲的是原始資料的索引，方便後續將decision score直接對應到原始資料做拓樸
    original_indices = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # 資料平衡處理
        smote = SMOTE(random_state=42, k_neighbors=3)
        enn = EditedNearestNeighbours(n_neighbors=3)
        smote_enn = SMOTEENN(smote=smote, enn=enn, random_state=random_state)
        X_resampled_train, y_resampled_train = smote_enn.fit_resample(X_train, y_train)

        # 建立線性支持向量機模型
        model = LinearSVC(random_state=random_state, max_iter=500000)

        # 超參數範圍
        parameters = {
            # 大C傾向完全擬和，可能導致過擬和；小C傾向正則化，可能增加訓練誤差但增加泛化能力
            'C': [1, 10, 100],
            # 目標是最大化分類邊界的距離，hinge是線性SVM的損失函數，squared_hinge是hinge的平方
            'loss': ['hinge']
        }

        # 使用 GridSearchCV 找最佳參數
        grid_search = GridSearchCV(model, parameters, cv=5, scoring='accuracy', n_jobs=n_jobs)
        grid_search.fit(X_resampled_train, y_resampled_train)
        best_model = grid_search.best_estimator_

        print(f"Best parameters for this fold: {grid_search.best_params_}")

        # 預測測試集
        decision_scores = best_model.decision_function(X_test)

        # 存儲測試結果
        y_true_all.extend(y_test)
        decision_scores_all.extend(decision_scores)
        original_indices.extend(test_index)

    return np.array(y_true_all), np.array(decision_scores_all), np.array(original_indices)



In [ ]:
import pickle
import time
import gc

# Define the data for each model
models = [
    # ("pass_outlier", pass_data_outlier_X, pass_data_outlier_y),
    # ("pass", pass_data_X, pass_data_y),
    # ("car_motor_outlier_overlap", car_motor_outlier_overlap_X, car_motor_outlier_overlap_y),
    # ("motor", motor_data_X, motor_data_y),
    ("car", car_data_X, car_data_y),
    ("full", full_data_X, full_data_y)
]

# Logistic
# for name, X, y in models:
#     print(f'{name} logistic start')
#     start_time = time.time()
#     y_log, decision_scores_log, indices_log = logistic_cm_kfold(X.astype(float), y)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     with open(f"./ModelPerformanceNewV3/{name}_performance_log.pkl", "wb") as f:
#         pickle.dump({
#             'y': y_log,
#             'decision_scores': decision_scores_log,
#             'indices': indices_log,
#             'elapsed_time': elapsed_time
#         }, f)
#     print(f'{name} logistic done in {elapsed_time:.2f} seconds')
#     del X, y, y_log, decision_scores_log
#     gc.collect()
    
# SVC
for name, X, y in models:
    print(f'{name} svc start')
    start_time = time.time()
    y_svc, decision_scores_svc, indices_svc = linear_svc_kfold(X.astype(float), y, n_jobs=14)
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(f"./ModelPerformanceNewV3/{name}_performance_svc.pkl", "wb") as f:
        pickle.dump({
            'y': y_svc,
            'decision_scores': decision_scores_svc,
            'indices': indices_svc,
            'elapsed_time': elapsed_time
        }, f)
    print(f'{name} svc done in {elapsed_time:.2f} seconds')
    del X, y, y_svc, decision_scores_svc
    gc.collect()

car svc start


KeyboardInterrupt: 